In [2]:
import pandas as pd
import plotly
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display

In [2]:
# df_agr = pd.read_csv('~/Documents/Energy Research/Value_of_Production_E_All_Area_Groups_NOFLAG.csv', encoding='latin1')
# df_emi = pd.read_csv('~/Documents/Energy Research/Emissions_Agriculture_Agriculture_total_E_All_Area_Groups_NOFLAG.csv', encoding='latin1')
# df_inp = pd.read_csv('~/Documents/Energy Research/Inputs_FertilizersNutrient_E_All_Area_Groups_NOFLAG.csv', encoding='latin1')

In [3]:
df_agr = pd.read_csv('Value_of_Production_E_All_Area_Groups_NOFLAG.csv', encoding='latin1')
df_emi = pd.read_csv('Emissions_Agriculture_Agriculture_total_E_All_Area_Groups_NOFLAG.csv', encoding='latin1')
df_inp = pd.read_csv('Inputs_FertilizersNutrient_E_All_Area_Groups_NOFLAG.csv', encoding='latin1')

In [4]:
df_agr = df_agr[df_agr['Element'] == 'Gross Production Value (constant 2014-2016 million US$)']
df_emi = df_emi[df_emi['Element'] == 'Emissions (CO2eq)']
df_inp = df_inp[df_inp['Element'] == 'Agricultural Use']

In [5]:
shared_id_vars = ['Area Code', 'Area', 'Item Code', 'Item', 'Element Code', 'Element', 'Unit']

In [5]:
df_agr = df_agr.melt(id_vars = shared_id_vars, var_name = "Year")
df_emi = df_emi.melt(id_vars = shared_id_vars, var_name = "Year")
df_inp = df_inp.melt(id_vars = shared_id_vars, var_name = "Year")

In [6]:
necessary_id_vars = ['Area', 'Item', 'Element', 'Unit', 'Year', 'value']
df_agr = df_agr[necessary_id_vars]
df_emi = df_emi[necessary_id_vars]
df_inp = df_inp[necessary_id_vars]

In [7]:
# Add subtotals
df_inp_subtotals = pd.DataFrame(columns=['Area', 'Item', 'Element', 'Unit', 'Year', 'value'])
for x, d in df_inp.groupby(['Area', 'Year']):
    subt = pd.DataFrame([[x[0], 'Total Fertilizer', 'Agricultural Use', 'tonnes', x[1], d.value.sum()]], columns=['Area', 'Item', 'Element', 'Unit', 'Year', 'value'])
    df_inp = df_inp.append(subt)

# Remove projections
df_emi = df_emi[df_emi.Year != 'Y2019']
df_emi = df_emi[df_emi.Year != 'Y2030']
df_emi = df_emi[df_emi.Year != 'Y2050']

In [8]:
# select only aggregates
df_agr = df_agr[(df_agr.Item == 'Agriculture (PIN)') | (df_agr.Item == 'Crops (PIN)') | (df_agr.Item == 'Livestock (PIN)')]

# change units from $mm to $s
df_agr['value'] = df_agr['value'].multiply(1000000)
df_agr['Element'] = 'Gross Production Value (constant 2014-2016 US$)'

In [9]:
df_combo = df_agr.merge(df_inp, on=['Area', 'Year'])

In [10]:
df_combo['Prod per inp'] = df_combo['value_x'].div(df_combo['value_y'], axis=0)

In [11]:
df_combo = df_combo.merge(df_emi, on=['Area', 'Year'])

In [12]:
df_combo['Prod per emi'] = df_combo['value_x'].div(df_combo['value'], axis=0)

In [13]:
# Begin plots

In [14]:
def unique_sorted_values(array):
    unique = array.unique().tolist()
    unique.sort()
    return unique

In [15]:
# Agriculture production plot

def common_filtering_ag_prod(area):
    plot_output_ag_prod.clear_output()
    
    common_filter_ag_prod = df_agr[df_agr.Area == area]
    
    with plot_output_ag_prod:
        fig_ag_prod = px.line(common_filter_ag_prod, x="Year", y="value", color='Item')
        fig_ag_prod.update_layout(yaxis_title='Production Value (2014-2016 USD)',
                      title='Agricultural Production Value')
        fig_ag_prod.show()

def dropdown_area_eventhandler(change):
    common_filtering_ag_prod(change.new)


plot_output_ag_prod = widgets.Output()

dropdown_area_ag_prod = widgets.Dropdown(options = unique_sorted_values(df_agr.Area))

# how to send dropdown_area_eventhandler with dataframe as argument so I don't need to write these functions for each plot?
dropdown_area_ag_prod.observe(dropdown_area_eventhandler, names='value') 
display(dropdown_area_ag_prod)
display(plot_output_ag_prod)

Dropdown(options=('Africa', 'Americas', 'Asia', 'Australia and New Zealand', 'Caribbean', 'Central America', '…

Output()

In [16]:
# Fertilizer use plot

def common_filtering_fert(area):
    plot_output_fert.clear_output()
    
    common_filter_fert = df_inp[df_inp.Area == area]
    
    with plot_output_fert:
        fig_fert = px.line(common_filter_fert, x="Year", y="value", color='Item')
        fig_fert.update_layout(yaxis_title='Fertilizer (tonnes)',
                      title='Agricultural Fertilizer Use')
        fig_fert.show()

def dropdown_area_eventhandler(change):
    common_filtering_fert(change.new)


plot_output_fert = widgets.Output()

dropdown_area_fert = widgets.Dropdown(options = unique_sorted_values(df_agr.Area))

# how to send dropdown_area_eventhandler with dataframe as argument so I don't need to write these functions for each plot?
dropdown_area_fert.observe(dropdown_area_eventhandler, names='value')
display(dropdown_area_fert)
display(plot_output_fert)

Dropdown(options=('Africa', 'Americas', 'Asia', 'Australia and New Zealand', 'Caribbean', 'Central America', '…

Output()

In [17]:
# Emissions plot

def common_filtering(area):
    plot_output.clear_output()
    
    common_filter = df_emi[df_emi.Area == area]
    
    with plot_output:
        fig = px.line(common_filter, x="Year", y="value", color='Item')
        fig.update_layout(yaxis_title='Emissions in CO2eq (gigagrams)',
                      title='Agricultural Emissions of CH4 and NO2 in CO2 Equivalent')
        fig.show()

def dropdown_area_eventhandler(change):
    common_filtering(change.new)


plot_output = widgets.Output()

dropdown_area = widgets.Dropdown(options = unique_sorted_values(df_agr.Area))

# how to send dropdown_area_eventhandler with dataframe as argument so I don't need to write these functions for each plot?
dropdown_area.observe(dropdown_area_eventhandler, names='value')
display(dropdown_area)
display(plot_output)

Dropdown(options=('Africa', 'Americas', 'Asia', 'Australia and New Zealand', 'Caribbean', 'Central America', '…

Output()